In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append("..")

In [2]:
orders_data_path = '../data/orders.json'
stock_data_path = '../data/stock.json'

with open(orders_data_path) as f:
    orders_data = json.load(f)
with open(stock_data_path) as f:
    stock_data = json.load(f)

In [3]:
{k : list(orders_data['two_recipes'].keys()).index(k)*2+2 for k in list(orders_data['two_recipes'].keys())}

{'two_portions': 2, 'four_portions': 4}

In [4]:
pd.DataFrame(orders_data).T

,four_portions,two_portions
two_recipes,13363,58684
three_recipes,22061,103886
four_recipes,18434,35154


In [5]:
pd.DataFrame(stock_data).T

,stock_count
recipe_1,3359
recipe_2,6502
recipe_3,1805
recipe_4,11217
recipe_5,1228
recipe_6,4370
recipe_7,12104
recipe_8,2304
recipe_9,9190
recipe_10,9149


In [6]:
stock_data

{'recipe_1': {'stock_count': 3359},
 'recipe_2': {'stock_count': 6502},
 'recipe_3': {'stock_count': 1805},
 'recipe_4': {'stock_count': 11217},
 'recipe_5': {'stock_count': 1228},
 'recipe_6': {'stock_count': 4370},
 'recipe_7': {'stock_count': 12104},
 'recipe_8': {'stock_count': 2304},
 'recipe_9': {'stock_count': 9190},
 'recipe_10': {'stock_count': 9149},
 'recipe_11': {'stock_count': 13370},
 'recipe_12': {'stock_count': 15490},
 'recipe_13': {'stock_count': 16334},
 'recipe_14': {'stock_count': 13140},
 'recipe_15': {'stock_count': 3659},
 'recipe_16': {'stock_count': 1280},
 'recipe_17': {'stock_count': 10059},
 'recipe_18': {'stock_count': 16849},
 'recipe_19': {'stock_count': 17576},
 'recipe_20': {'stock_count': 4404},
 'recipe_21': {'stock_count': 17933},
 'recipe_22': {'stock_count': 23955}}

In [7]:
sum([ i['stock_count'] for i in stock_data.values() ])

215277

In [8]:
sum([j for i in orders_data.values() for j in i.values()])

251582

In [9]:
sorted([ i['stock_count'] for i in stock_data.values() ])

[1228,
 1280,
 1805,
 2304,
 3359,
 3659,
 4370,
 4404,
 6502,
 9149,
 9190,
 10059,
 11217,
 12104,
 13140,
 13370,
 15490,
 16334,
 16849,
 17576,
 17933,
 23955]

In [10]:
def fetch_portion(stock_list, num_recipe, num_portion, quantity):
    sorted_stock_list = np.array(sorted(stock_list))
    stock_index=0
    while quantity > 0:
        if len(sorted_stock_list) < stock_index+num_recipe:
            return None
        to_fetch = min(quantity*num_portion, sorted_stock_list[stock_index] - sorted_stock_list[stock_index] % num_portion)
        sorted_stock_list[stock_index:stock_index+num_recipe] = sorted_stock_list[stock_index:stock_index+num_recipe] - to_fetch
        quantity = quantity - (to_fetch / num_portion)
        stock_index += 1
    
    return sorted_stock_list

In [11]:
def r_to_num(r):
    return {'two_recipes': 2, 'three_recipes': 3, 'four_recipes': 4}.get(r)
    
def p_to_num(p):
    return {'two_portions': 2, 'four_portions': 4}.get(p)

In [12]:
recipe_portion_quantity_tuple = [(r_to_num(r), p_to_num(p), orders_data[r][p]) for r in orders_data.keys() for p in orders_data[r].keys()]
recipe_portion_quantity_tuple = sorted(recipe_portion_quantity_tuple, key=lambda c : (c[0], c[1]), reverse = True)
print(recipe_portion_quantity_tuple)

[(4, 4, 18434), (4, 2, 35154), (3, 4, 22061), (3, 2, 103886), (2, 4, 13363), (2, 2, 58684)]


In [13]:
recipe_portion_quantity_tuple = [(4, 4, 1000) , (4,2, 360), (3,4, 1230), (3,2, 5430), (2,4, 9870), (2,2, 10000)]

In [14]:
stock_list = [ i['stock_count'] for i in stock_data.values() ]
print(stock_list)

[3359, 6502, 1805, 11217, 1228, 4370, 12104, 2304, 9190, 9149, 13370, 15490, 16334, 13140, 3659, 1280, 10059, 16849, 17576, 4404, 17933, 23955]


In [15]:
print(sorted(stock_list))
for r, p, q in tqdm(recipe_portion_quantity_tuple):
    stock_list = fetch_portion(stock_list, r, p, q)
    if stock_list is None:
        print(False)
        break
    print(sorted(stock_list))
if stock_list is not None:
    print(True)

100%|██████████| 6/6 [00:00<00:00, 1033.08it/s]

[1228, 1280, 1805, 2304, 3359, 3659, 4370, 4404, 6502, 9149, 9190, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
[0, 0, 0, 1, 587, 939, 2174, 2708, 6502, 9149, 9190, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
[0, 0, 0, 1, 1, 219, 1454, 1988, 6368, 9149, 9190, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
[0, 0, 0, 0, 1, 1, 2, 3, 1664, 5681, 6258, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 863, 6039, 7504, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 12033, 16160, 17933, 23955]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 9969, 20121]
True


In [16]:
sum([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 9969, 20121])

30097

In [17]:
sum([r*p*q for r, p, q in recipe_portion_quantity_tuple])

185180

In [18]:
sum([ i['stock_count'] for i in stock_data.values() ])

215277

In [19]:
215277 - 185180

30097

In [20]:
from recipe_allocator import defaults_recipe_allocation
defaults_recipe_allocation(orders_data, stock_data)

False

In [21]:
orders_data

{'two_recipes': {'two_portions': 58684, 'four_portions': 13363},
 'three_recipes': {'two_portions': 103886, 'four_portions': 22061},
 'four_recipes': {'two_portions': 35154, 'four_portions': 18434}}

In [22]:
orders_data = {'two_recipes': {'two_portions': 10000, 'four_portions': 9870},
 'three_recipes': {'two_portions': 5430, 'four_portions': 1230},
 'four_recipes': {'two_portions': 360, 'four_portions': 1000}}

In [23]:
defaults_recipe_allocation(orders_data, stock_data)

True

In [24]:
defaults_recipe_allocation(orders_data, stock_data, debug=1)

recipe stock list after iteration 1
iteration 1 = 4 recipes, 4 portions, 1000 customers
[0, 0, 0, 1, 587, 939, 2174, 2708, 6502, 9149, 9190, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
recipe stock list after iteration 2
iteration 2 = 4 recipes, 2 portions, 360 customers
[0, 0, 0, 1, 1, 219, 1454, 1988, 6368, 9149, 9190, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
recipe stock list after iteration 3
iteration 3 = 3 recipes, 4 portions, 1230 customers
[0, 0, 0, 0, 1, 1, 2, 3, 1664, 5681, 6258, 10059, 11217, 12104, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
recipe stock list after iteration 4
iteration 4 = 3 recipes, 2 portions, 5430 customers
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 863, 6039, 7504, 13140, 13370, 15490, 16334, 16849, 17576, 17933, 23955]
recipe stock list after iteration 5
iteration 5 = 2 recipes, 4 portions, 9870 customers
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 12033, 16160, 17933, 

True